**Bitcoin price prediction with Bayesian Neural Network Regression using torchBNN and PyTorch.**

In [84]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import datetime
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchinfo import summary
import torchbnn as bnn

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [85]:
cwd = os.getcwd()
from os.path import dirname, abspath
while not cwd.endswith('BT4222_repo'):
    cwd = os.path.dirname(cwd)

In [86]:
PATH = os.path.join(cwd,'data','cooked_data','cooked_complete_dataset.csv')

In [87]:
df = pd.read_csv(PATH)
df.dropna(inplace = True)
df.head()

,date,Adj_Close_BTC-USD,Open_BTC-USD,High_BTC-USD,Low_BTC-USD,Volume_BTC-USD,Adj_Close_SPY,Adj_Close_GLD,Adj_Close_CHFUSD=X,Adj_Close_CNYUSD=X,Adj_Close_EURUSD=X,Adj_Close_GBPUSD=X,Adj_Close_JPYUSD=X,coindesk_sentiment,num_of_coindesk_posts,reddit_comments_sentiments,top_50_reddit_posts_sentiments,blockchain_transactions_per_block,blockchain_hash_rates
1,14/12/20,19246.64453,19144.49219,19305.09961,19012.70898,2.247400e+10,361.926788,171.539993,1.125442,0.152772,1.213340,1.331824,0.009621,0.249489,12.0,0.158060,0.677618,2167.931034,134574371.4
2,15/12/20,19417.07617,19246.91992,19525.00781,19079.84180,2.674198e+10,366.819824,173.940002,1.127930,0.152679,1.214890,1.333084,0.009614,0.173773,18.0,0.101930,0.447277,2288.857143,129933875.8
3,16/12/20,21310.59766,19418.81836,21458.90820,19298.31641,4.440901e+10,367.395508,174.899994,1.129382,0.152945,1.215430,1.344447,0.009649,0.341491,11.0,0.127344,0.480809,2204.314685,132718173.2
4,17/12/20,22805.16211,21308.35156,23642.66016,21234.67578,7.137861e+10,369.449982,176.740005,1.129446,0.153109,1.219959,1.350293,0.009664,0.197572,10.0,0.135945,0.539729,2399.077519,119724785.6
5,18/12/20,23137.96094,22806.79688,23238.60156,22399.81250,4.038790e+10,367.974792,176.440002,1.130301,0.153090,1.226272,1.357018,0.009696,0.315601,2.0,0.135441,0.449503,2392.031847,145711560.8


In [88]:
df['date'] = df['date'].apply(lambda x: datetime.datetime.strptime(x, "%d/%m/%y"))
df["date"] = pd.to_datetime(df["date"], format='%d/%m/%Y', infer_datetime_format=True)

**Helper Functions for creating lags and scaling**

In [180]:
def lag(data, dic):
    cols = []
    for key, value in dic.items():
        for i in range(1, value+1):
            cols.append(data[key].shift(i).rename('{}_lag{}'.format(data[key].name, i)))
    return pd.concat([data["date"],data["Adj_Close_BTC-USD"]] + cols, axis = 1)

scaler = MinMaxScaler()

def scale_and_convert_to_tensor(Xtrain, Xtest, Ytrain, Ytest, scaleTarget = False):  
    global scaler
    
    # Standardise features
    Xtrain_standardised = scaler.fit_transform(Xtrain)
    Xtest_standardised = scaler.transform(Xtest)
    
    # Standardise target
    Ytrain_standardised = Ytrain
    Ytest_standardised = Ytest
    
    if scaleTarget:    
        Ytrain_standardised = scaler.fit_transform(np.array(Ytrain).reshape(-1, 1))
        Ytest_standardised = scaler.transform(np.array(Ytest).reshape(-1, 1))
    
    ## Change to tensor
    Xtrain_tensor = torch.from_numpy(Xtrain_standardised).float()
    Ytrain_tensor = torch.from_numpy(np.array(Ytrain_standardised)).float()
    Xtest_tensor = torch.from_numpy(Xtest_standardised).float()
    Ytest_tensor = torch.from_numpy(np.array(Ytest_standardised)).float()
        
    return (Xtrain_tensor, Xtest_tensor, Ytrain_tensor, Ytest_tensor)

## Without Sentiments

**1. Create feature lags**

In [90]:
feature_lags = {"Adj_Close_BTC-USD" : 1,
                "Adj_Close_SPY" : 1,
                "Adj_Close_GLD" : 1,
                "Adj_Close_CHFUSD=X" : 1,
                "Adj_Close_EURUSD=X" : 1,
                "Adj_Close_GBPUSD=X" : 1,
                "Adj_Close_JPYUSD=X" : 1,
                "blockchain_transactions_per_block" : 1,
                "blockchain_hash_rates" : 1}

data = lag(df, feature_lags).drop_duplicates(subset = 'date')

**2. Handle train-test split**

In [91]:
data = data[(data["date"] >= "2021-01-01") & (data["date"] <= "2021-04-12")]

## train, validation and test split
train = data.loc[(data["date"] >= "2021-01-01") & (data["date"] <= "2021-03-14")]
validation = data[(data["date"] >= "2021-03-15") & (data["date"] <= "2021-03-29")]
test = data[(data["date"] >= "2021-03-30") & (data["date"] <= "2021-04-12")]

## refit and full for later use 
refit = data[(data["date"] >= "2021-01-01") & (data["date"] <= "2021-03-29")]
full = data.copy(deep = True)

## train 
X_train = train.drop(["date", "Adj_Close_BTC-USD"], axis = 1)
y_train = train["Adj_Close_BTC-USD"]

## validation
X_val = validation.drop(["date", "Adj_Close_BTC-USD"], axis = 1)
y_val = validation["Adj_Close_BTC-USD"]

## test
X_test = test.drop(["date", "Adj_Close_BTC-USD"], axis = 1)
y_test = test["Adj_Close_BTC-USD"]

## refit
X_refit = refit.drop(["date", "Adj_Close_BTC-USD"], axis = 1)
y_refit = refit["Adj_Close_BTC-USD"]

## full
X_full = full.drop(["date", "Adj_Close_BTC-USD"], axis = 1)
y_full = full["Adj_Close_BTC-USD"]

**3. Standardise dataset and transform into tensors for pytorch**

In [92]:
## Standardise datasets and convert into tensors
Xtrain_tensor, Xval_tensor, Ytrain_tensor, Yval_tensor = scale_and_convert_to_tensor(X_train, X_val, y_train, y_val, scaleTarget = True)
_, Xtest_tensor, _, Ytest_tensor = scale_and_convert_to_tensor(X_train, X_test, y_train, y_test, scaleTarget = True)

print("X train tensor",Xtrain_tensor.shape)
print("Y train tensor",Ytrain_tensor.shape)

print("X val tensor",Xval_tensor.shape)
print("Y val tensor",Yval_tensor.shape)

print("X test tensor",Xtest_tensor.shape)
print("Y test tensor",Ytest_tensor.shape)

X train tensor torch.Size([73, 9])
Y train tensor torch.Size([73, 1])
X val tensor torch.Size([15, 9])
Y val tensor torch.Size([15, 1])
X test tensor torch.Size([14, 9])
Y test tensor torch.Size([14, 1])


**4. Define BNN training and evaluation pipeline**

In [68]:
def train_model_and_evaluate_regression(Xtrain_tensor, Ytrain_tensor, Xtest_tensor, Ytest_tensor, layers = [32,8], learning_param = 0.01, kl_weight = 0.01, steps = 100, printStep = True):    
    in_features = Xtrain_tensor.shape[1]
    batch_size = Xtrain_tensor.shape[0]
    
    ## Ensure reproducibility
    seed = 1
    torch.manual_seed(seed)
    
    # Build model
    layer = []
    
    ## Input layer
    layer.append(bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features = in_features, out_features = layers[0]))
    layer.append(nn.ReLU())
    
    ## Hidden layers
    for index, neurons in enumerate(layers):
        if index != (len(layers)-1):
            layer.append(bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=neurons, out_features=layers[index+1]))
            layer.append(nn.ReLU())

    ## Output layer
    layer.append(bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=layers[-1], out_features=1))
    
    model = nn.Sequential(*layer)
    # Define Loss
    mse_loss = nn.MSELoss()
    kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)

    ## Define optimiser with learning rate
    optimizer = optim.Adam(model.parameters(), lr = learning_param)
    
    ### Train model
    for step in range(steps):
        torch.manual_seed(seed)
        
        pre = model(Xtrain_tensor)
        mse = mse_loss(pre, Ytrain_tensor)
        kl = kl_loss(model)
        cost = mse + kl_weight*kl

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        # Print Progress
        if step%50==0 and printStep:
            print('[Step %d]: MSE : %.8f, KL : %.8f' % (step , mse.item(), kl.item()))
            
    train_mse = mse.item()
    kl_loss = kl.item()
    
    ## Predict Test
    torch.manual_seed(seed)
    y_predict = model(Xtest_tensor)
    
    ## Performance Evaluation on test - MSE
    test_mse = mean_squared_error(Ytest_tensor.detach().numpy(),y_predict.detach().numpy())
    
    ## Inverse Standard Scaler - for unscaled RMSE 
    y_actual = scaler.inverse_transform(Ytest_tensor.detach().numpy().reshape(-1, 1))
    y_predict = scaler.inverse_transform(y_predict.detach().numpy().reshape(-1, 1))
    test_rmse = math.sqrt(mean_squared_error(y_actual,y_predict))
    
    return (model, y_predict, train_mse, test_mse, test_rmse, kl_loss)

**6. Perform Grid Search**

Perform grid search and evaluate based on validation set.

In [55]:
training_mse_list = []
val_mse_list = []
val_rmse_list = []
kl_list = []
combination = []

learning_param_list = pd.Series(np.linspace(0.001,0.5,50)).apply(lambda x: round(x,3))
kl_weight =  pd.Series(np.linspace(0.001,0.5,50)).apply(lambda x: round(x,3))
layers_list = ([32,8],[32,16],[32,16,8])

for layer in layers_list:
    print("--- Layer: ", layer)
    for lr in learning_param_list:
        print("-- Learning Param: ", lr)
        for kl in kl_weight:
            combination.append("layer: {} lr: {} kl: {}".format(layer,lr,kl))
            _ ,_, train_mse, val_mse, val_rmse, kl_loss = train_model_and_evaluate_regression(Xtrain_tensor, Ytrain_tensor, Xval_tensor, Yval_tensor, layer, learning_param = lr, kl_weight = kl, steps = 100, printStep = False)
            training_mse_list.append(train_mse)
            val_mse_list.append(val_mse)
            val_rmse_list.append(val_rmse)
            kl_list.append(kl_loss)
            
print("Complete")

--- Layer:  [32, 8]
-- Learning Param:  0.001
-- Learning Param:  0.011
-- Learning Param:  0.021
-- Learning Param:  0.032
-- Learning Param:  0.042
-- Learning Param:  0.052
-- Learning Param:  0.062
-- Learning Param:  0.072
-- Learning Param:  0.082
-- Learning Param:  0.093
-- Learning Param:  0.103
-- Learning Param:  0.113
-- Learning Param:  0.123
-- Learning Param:  0.133
-- Learning Param:  0.144
-- Learning Param:  0.154
-- Learning Param:  0.164
-- Learning Param:  0.174
-- Learning Param:  0.184
-- Learning Param:  0.194
-- Learning Param:  0.205
-- Learning Param:  0.215
-- Learning Param:  0.225
-- Learning Param:  0.235
-- Learning Param:  0.245
-- Learning Param:  0.256
-- Learning Param:  0.266
-- Learning Param:  0.276
-- Learning Param:  0.286
-- Learning Param:  0.296
-- Learning Param:  0.307
-- Learning Param:  0.317
-- Learning Param:  0.327
-- Learning Param:  0.337
-- Learning Param:  0.347
-- Learning Param:  0.357
-- Learning Param:  0.368
-- Learning Param:

In [102]:
results = pd.DataFrame({"Combination": combination,"Train MSE":training_mse_list, "Val MSE":val_mse_list, "Val RMSE":  val_rmse_list,"KL Loss":kl_list})
results.to_csv("Combinations_regression_withoutSentiments&lagged2.csv")

## Find the hyperparameters with gives the lowest test RMSE
results[results['Val RMSE'] ==  results['Val RMSE'].min()]['Combination']

layer: [32, 16, 8] lr: 0.123 kl: 0.174


In [94]:
layer = [32, 16, 8]
lr = 0.123
kl = 0.154

model, y_predict, _, _, test_rmse, kl_loss = train_model_and_evaluate_regression(Xtrain_tensor, Ytrain_tensor, Xval_tensor, Yval_tensor, layer, learning_param = lr, kl_weight = kl, steps = 100, printStep = False)
print("Val RMSE: ",test_rmse)

Val RMSE:  1289.6513579258544


**7. Retrain the model with selected hyperparameters and all train data available**

In [95]:
### Helper function for retraining the model with all data available
def train_model_and_predict(Xtrain_tensor, Ytrain_tensor, layers = [32,8], learning_param = 0.01, kl_weight = 0.01, steps = 100):    
    """ 
    Trains model and returns predictions on entire dataset.
    """
    in_features = Xtrain_tensor.shape[1]
    batch_size = Xtrain_tensor.shape[0]
    
    ## Ensure reproducibility
    seed = 1
    torch.manual_seed(seed)
    
    # Build model
    layer = []
    
    ## Input layer
    layer.append(bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features = in_features, out_features = layers[0]))
    layer.append(nn.ReLU())
    
    ## Hidden layers
    for index, neurons in enumerate(layers):
        if index != (len(layers)-1):
            layer.append(bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=neurons, out_features=layers[index+1]))
            layer.append(nn.ReLU())

    ## Output layer
    layer.append(bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=layers[-1], out_features=1))
    
    model = nn.Sequential(*layer)
    # Define Loss
    mse_loss = nn.MSELoss()
    kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)

    ## Define optimiser with learning rate
    optimizer = optim.Adam(model.parameters(), lr = learning_param)
    
    ### Train model
    for step in range(steps):
        torch.manual_seed(seed)
        
        pre = model(Xtrain_tensor)
        mse = mse_loss(pre, Ytrain_tensor)
        kl = kl_loss(model)
        cost = mse + kl_weight*kl

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
    ## Predict Test
    torch.manual_seed(seed)
    y_predict = model(Xtrain_tensor)

    ## Inverse Standard Scaler
    y_predict = scaler.inverse_transform(y_predict.detach().numpy().reshape(-1, 1))
    
    return (model,y_predict)

In [96]:
### Standardise refit data (train + val)
Xrefit_tensor, _, Yrefit_tensor, _ = scale_and_convert_to_tensor(X_refit, X_refit, y_refit, y_refit, scaleTarget = True)
print("X refit tensor",Xrefit_tensor.shape)
print("Y refit tensor",Yrefit_tensor.shape)

X refit tensor torch.Size([88, 9])
Y refit tensor torch.Size([88, 1])


In [97]:
### Train on refit data and evaluate on test
_, y_predict, _, _, test_rmse, kl_loss = train_model_and_evaluate_regression(Xrefit_tensor, Yrefit_tensor, Xtest_tensor, Ytest_tensor, layer, learning_param = lr, kl_weight = kl, steps = 100, printStep = False)
print("Test RMSE: ",test_rmse)
print(y_predict)

Test RMSE:  3140.482287802305
[[55760.875]
 [55758.062]
 [55762.164]
 [55776.81 ]
 [55776.676]
 [55777.035]
 [55776.32 ]
 [55786.953]
 [55778.188]
 [55776.98 ]
 [55770.79 ]
 [55781.625]
 [55783.836]
 [55783.79 ]]


**8. Obtain predictions for Backtesting**

In [98]:
Xfull_tensor, _, Yfull_tensor, _ = scale_and_convert_to_tensor(X_full, X_full, y_full, y_full, scaleTarget = True)
print("X test tensor",Xfull_tensor.shape)
print("Y test tensor",Yfull_tensor.shape)

X test tensor torch.Size([102, 9])
Y test tensor torch.Size([102, 1])


In [99]:
model, y_predict = train_model_and_predict(Xfull_tensor, Yfull_tensor, layers = layer, learning_param = lr, kl_weight = kl, steps = 100)        
math.sqrt(mean_squared_error(y_full,y_predict))

2323.6570432281137

In [17]:
#pd.DataFrame(y_predict).to_csv("out.csv")

## With Sentiments

**1. Create Feature Lags**

In [183]:
feature_lags = {"Adj_Close_BTC-USD" : 1, 
                "Adj_Close_SPY" : 1,
                "Adj_Close_GLD" : 1,
                "Adj_Close_CHFUSD=X" : 1,
                "Adj_Close_EURUSD=X" : 1,
                "Adj_Close_GBPUSD=X" : 1,
                "Adj_Close_JPYUSD=X" : 1,
                "blockchain_transactions_per_block" : 1,
                "blockchain_hash_rates" : 1,
                "coindesk_sentiment" : 1,
                "reddit_comments_sentiments" : 1,
                "top_50_reddit_posts_sentiments" : 1}

data = lag(df, feature_lags).drop_duplicates(subset = 'date')

**2. Handle Train-test split**

In [184]:
data = data[(data["date"] >= "2021-01-01") & (data["date"] <= "2021-04-12")]

## train, validation and test split
train = data.loc[(data["date"] >= "2021-01-01") & (data["date"] <= "2021-03-14")]
validation = data[(data["date"] >= "2021-03-15") & (data["date"] <= "2021-03-29")]
test = data[(data["date"] >= "2021-03-30") & (data["date"] <= "2021-04-12")]

## refit and full for later use 
refit = data[(data["date"] >= "2021-01-01") & (data["date"] <= "2021-03-29")]
full = data.copy(deep = True)

## train 
X_train = train.drop(["date", "Adj_Close_BTC-USD"], axis = 1)
y_train = train["Adj_Close_BTC-USD"]

## validation
X_val = validation.drop(["date", "Adj_Close_BTC-USD"], axis = 1)
y_val = validation["Adj_Close_BTC-USD"]

## test
X_test = test.drop(["date", "Adj_Close_BTC-USD"], axis = 1)
y_test = test["Adj_Close_BTC-USD"]

## refit
X_refit = refit.drop(["date", "Adj_Close_BTC-USD"], axis = 1)
y_refit = refit["Adj_Close_BTC-USD"]

## full
X_full = full.drop(["date", "Adj_Close_BTC-USD"], axis = 1)
y_full = full["Adj_Close_BTC-USD"]

**3. Standardise dataset and transform to tensors** 

In [185]:
## Standardise datasets and convert into tensors
Xtrain_tensor, Xval_tensor, Ytrain_tensor, Yval_tensor = scale_and_convert_to_tensor(X_train, X_val, y_train, y_val, scaleTarget = True)
_, Xtest_tensor, _, Ytest_tensor = scale_and_convert_to_tensor(X_train, X_test, y_train, y_test, scaleTarget = True)

print("X train tensor",Xtrain_tensor.shape)
print("Y train tensor",Ytrain_tensor.shape)

print("X val tensor",Xval_tensor.shape)
print("Y val tensor",Yval_tensor.shape)

print("X test tensor",Xtest_tensor.shape)
print("Y test tensor",Ytest_tensor.shape)

X train tensor torch.Size([73, 12])
Y train tensor torch.Size([73, 1])
X val tensor torch.Size([15, 12])
Y val tensor torch.Size([15, 1])
X test tensor torch.Size([14, 12])
Y test tensor torch.Size([14, 1])


**4. Perform GridSearch**

In [14]:
training_mse_list = []
val_mse_list = []
val_rmse_list = []
kl_list = []
combination = []

learning_param_list = pd.Series(np.linspace(0.001,0.5,50)).apply(lambda x: round(x,3))
kl_weight =  pd.Series(np.linspace(0.001,0.5,50)).apply(lambda x: round(x,3))
layers_list = ([32,8],[32,16],[32,16,8])

for layer in layers_list:
    print("--- Layer: ", layer)
    for lr in learning_param_list:
        print("-- Learning Param: ", lr)
        for kl in kl_weight:
            combination.append("layer: {} lr: {} kl: {}".format(layer,lr,kl))
            _ ,_, train_mse, val_mse, val_rmse, kl_loss = train_model_and_evaluate_regression(Xtrain_tensor, Ytrain_tensor, Xval_tensor, Yval_tensor, layer, learning_param = lr, kl_weight = kl, steps = 100, printStep = False)
            training_mse_list.append(train_mse)
            val_mse_list.append(val_mse)
            val_rmse_list.append(val_rmse)
            kl_list.append(kl_loss)
            
print("Complete")

--- Layer:  [32, 8]
-- Learning Param:  0.001
-- Learning Param:  0.011
-- Learning Param:  0.021
-- Learning Param:  0.032
-- Learning Param:  0.042
-- Learning Param:  0.052
-- Learning Param:  0.062
-- Learning Param:  0.072
-- Learning Param:  0.082
-- Learning Param:  0.093
-- Learning Param:  0.103
-- Learning Param:  0.113
-- Learning Param:  0.123
-- Learning Param:  0.133
-- Learning Param:  0.144
-- Learning Param:  0.154
-- Learning Param:  0.164
-- Learning Param:  0.174
-- Learning Param:  0.184
-- Learning Param:  0.194
-- Learning Param:  0.205
-- Learning Param:  0.215
-- Learning Param:  0.225
-- Learning Param:  0.235
-- Learning Param:  0.245
-- Learning Param:  0.256
-- Learning Param:  0.266
-- Learning Param:  0.276
-- Learning Param:  0.286
-- Learning Param:  0.296
-- Learning Param:  0.307
-- Learning Param:  0.317
-- Learning Param:  0.327
-- Learning Param:  0.337
-- Learning Param:  0.347
-- Learning Param:  0.357
-- Learning Param:  0.368
-- Learning Param:

In [193]:
results = pd.DataFrame({"Combination": combination,"Train MSE":training_mse_list, "Val MSE":val_mse_list, "Val RMSE":  val_rmse_list,"KL Loss":kl_list})
results.to_csv("Combinations_regression_withSentiments&lagged2.csv")

## Find the hyperparameters with gives the lowest test RMSE
results[results['Val RMSE'] ==  results['Val RMSE'].min()]['Combination']

4825    layer: [32, 16] lr: 0.469 kl: 0.256
Name: Combination, dtype: object

In [219]:
layer = [32, 16]
lr =  0.469
kl = 0.256

model, y_predict, _, _, test_rmse, kl_loss = train_model_and_evaluate_regression(Xtrain_tensor, Ytrain_tensor, Xval_tensor, Yval_tensor, layer, learning_param = lr, kl_weight = kl, steps = 100, printStep = False)
print("Val RMSE: ",test_rmse)

Val RMSE:  1260.3213875833417


**5. Retrain the model with selected hyperparameters and all data available.**

In [220]:
### Standardise refit data (train + val)
Xrefit_tensor, _, Yrefit_tensor, _ = scale_and_convert_to_tensor(X_refit, X_refit, y_refit, y_refit, scaleTarget = True)
print("X refit tensor",Xrefit_tensor.shape)
print("Y refit tensor",Yrefit_tensor.shape)

X refit tensor torch.Size([88, 12])
Y refit tensor torch.Size([88, 1])


In [221]:
### Train on refit data and evaluate on test
_, y_predict, _, _, test_rmse, kl_loss = train_model_and_evaluate_regression(Xrefit_tensor, Yrefit_tensor, Xtest_tensor, Ytest_tensor, layers = layer, learning_param = lr, kl_weight = kl, steps = 100, printStep = False)
print("Test RMSE: ", test_rmse)
print(y_predict)

Test RMSE:  7158.665797479304
[[47359.727]
 [47295.473]
 [53280.14 ]
 [47736.695]
 [48592.508]
 [53786.812]
 [47595.332]
 [58345.59 ]
 [60096.895]
 [57160.02 ]
 [54645.5  ]
 [60626.598]
 [61218.37 ]
 [56808.38 ]]


**6. Obtain predictions for backtest**

In [222]:
Xfull_tensor, _, Yfull_tensor, _ = scale_and_convert_to_tensor(X_full, X_full, y_full, y_full, scaleTarget = True)
print("X test tensor",Xfull_tensor.shape)
print("Y test tensor",Yfull_tensor.shape)

X test tensor torch.Size([102, 12])
Y test tensor torch.Size([102, 1])


In [223]:
model, y_predict = train_model_and_predict(Xfull_tensor, Yfull_tensor, layers = layer, learning_param = lr, kl_weight = kl, steps = 100)        
math.sqrt(mean_squared_error(y_full,y_predict))

9967.97182503028

In [224]:
#pd.DataFrame(y_predict).to_csv("out.csv")